In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
from datetime import datetime
import os
import tabula
from zipfile import ZipFile
import glob

In [2]:
start_time = datetime.now()

In [3]:
dir = 'Y:\\delete\\US EPA'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

In [4]:
ser = Service("C:/Users/AIMPAI0/Downloads/chromedriver.exe")
op = webdriver.ChromeOptions()
prefs = {"download.default_directory" : "Y:\\delete\\US EPA","plugins.always_open_pdf_externally": True}
op.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(service=ser, options=op)
driver.maximize_window()  
driver.get("https://www.epa.gov/tsca-inventory/how-access-tsca-inventory#download")
driver.implicitly_wait(5)
driver.find_element(by=By.XPATH, value='//*[@id="main"]/div/div[1]/div[2]/div[1]/article/div[2]/div/p[6]/a').click()
time.sleep(10)
driver.close()

In [5]:
file_name = "Y:/delete/US EPA/csv-non-cbi-tsca-inventory-022022.zip"
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
  
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall('Y:/delete/US EPA')
    print('Done!')

File Name                                             Modified             Size
TSCAINV_202202.csv                             2022-02-15 10:18:24      9286342
PMNACC_202202.csv                              2022-02-15 10:13:58      2021763
Extracting all the files now...
Done!


In [11]:
path = 'Y:\\delete\\US EPA'
csv_files = glob.glob(os.path.join(path, "TSCA*.csv"))
for f in csv_files:
    dataset = pd.read_csv(f)
dataset = dataset[['CASRN']]
dataset = dataset.applymap(lambda x: x.strip() if isinstance(x, str) else x)
dataset.rename(columns = {'CASRN':'CAS Number'}, inplace = True)

CAS = pd.read_excel('Y:/delete/Chemical CAS.xlsx')
CAS['value'] = 'Concluded'
CAS = CAS.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [20]:
result = pd.merge(dataset, CAS, how="outer", on="CAS Number")

In [22]:
result['Jurisdiction'] = 'US'
result['RegulatoryAgency'] = 'Environment Protection Aganecy (EPA)'
result['TypeOfRegulation'] = 'Trade'
result['Criteria'] = 'Relevant regulatory flag'

result['Description'] = 'TSCA flag relating to import/export'
result.loc[result['value'] == 'Concluded', 'Description'] = 'No relevantTSCA flag'

result['HazardClassification'] = 'High'
result.loc[result['value'] == 'Concluded', 'HazardClassification'] = 'Low'

result['CauseOfClassification'] = 'Has TCSA flag S,PMN, TP,SP,T,R,5E,5F,12C'
result.loc[result['value'] == 'Concluded', 'CauseOfClassification'] = 'Other TSCA flag or no TSCA flag'

result['SourceName'] = 'US EPA'
result['SourceURL'] = 'https://www.epa.gov/tsca-inventory/how-access-tsca-inventory#download'

result.head(2)

,CAS Number,value,Jurisdiction,RegulatoryAgency,TypeOfRegulation,Criteria,Description,HazardClassification,CauseOfClassification,SourceName,SourceURL
0,50-00-0,NaN,US,Environment Protection Aganecy (EPA),Trade,Relevant regulatory flag,TSCA flag relating to import/export,High,"Has TCSA flag S,PMN, TP,SP,T,R,5E,5F,12C",US EPA,https://www.epa.gov/tsca-inventory/how-access-...
1,50-01-1,NaN,US,Environment Protection Aganecy (EPA),Trade,Relevant regulatory flag,TSCA flag relating to import/export,High,"Has TCSA flag S,PMN, TP,SP,T,R,5E,5F,12C",US EPA,https://www.epa.gov/tsca-inventory/how-access-...


In [ ]:
result.to_csv('Y:/delete/Reugulatory Assessment.csv',index=False,mode='a',header=False)

In [23]:
end_time = datetime.now()
print('The total time taken is', (end_time - start_time))

The total time taken is 0:09:49.887757
